# Comp 551-Assignment 3


## Aymen Rumi-260661663
   
### ***-Refer to report for further detailed exlanation/observation/answers to each inidivudal question and their subquestions: this file contains mainly code and ouput values that have taken a very long time to run-***

In [5]:
import numpy as np
import pandas as pd
import string
import csv
from collections import Counter
from operator import itemgetter
import itertools
from scipy import sparse
import re
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")



# Question 1: Convert Dataset & Create Vocab List

## The code below contains all method used to convert text file into, vocab table, translation into numerical table, conversion to binary bag of words representation, and conversion to frequency bag of words

In [6]:
# function given text file,removes all punctuation, uppercase
# and returns an array value of the word with the number id and frequency in the text
def vocabulary_list(csv_file, cutoff):  
    # initializing punctuations and various other characters for removal
    punctuations = str.maketrans(dict.fromkeys(string.punctuation))
    removals = re.compile(r'<[^>]+>')
    temporary = Counter()    
    #iterate through each row in csv file, each row corresponds to a review
    for index,row in csv_file.iterrows():
        # change all uppercase to lowercase, remove punctuation and remove non-ascii characters     
        review = row[0].lower().translate(punctuations).encode('ascii', errors='ignore').decode()
        # remove HTML tags
        review = removals.sub('', review)
        count = Counter(review.split())
        temporary = temporary + count
    table = []
    counter = 1
    #return a table for sorted vocabulary words for up tp cutoff number
    for word, value in sorted(temporary.items(), key=itemgetter(1), reverse=True):
        table.append((word, counter, value))
        counter = counter+1
        if counter > cutoff:
            return table
    while counter <= cutoff:
        table.append(("0", counter, 0))
        counter += 1
    return table


def column(array, col):
        return [row[col] for row in array]

#given a text file, it replaces all the words in the review with it's numerical correspondance
def text_translation(csv_file, vocab):
    # initializing punctuations and various other characters for removal
    punctuations = str.maketrans(dict.fromkeys(string.punctuation))
    printable_table = str.maketrans(dict.fromkeys(string.printable))
    removals = re.compile(r'<[^>]+>')
    words = column(vocab, 0)
    numbers = column(vocab, 1)
    for index, row in csv_file.iterrows():       
        review = row[0].lower().translate(punctuations)
        review_copy = review
        for word in review_copy.split(' '):
            try:
                review = re.sub(r"\b%s\b" % word, str(numbers[words.index(word)]), review)
            except ValueError:
                review = re.sub(r"\b%s\b" % word, "", review)
        csv_file.iat[index, 0] = review
    return csv_file


def makeFile(fileName, data):
    file = open(fileName, 'w')   
    for line in data:
        file.write('\t'.join(str(s) for s in line) + '\n')                          
    file.close()   
    return file

def makeTranslation(file,vocabFile, targetFile, vocabulary):
    vocab=open(vocabFile, 'r')
    vocab_table=[]
    with open(vocabFile) as input:
        for line in input:
            temp = line.split('\t')
            vocab_table.append([temp[0], int(temp[1]), int(temp[2])])
    pd.DataFrame.to_csv(text_translation(file, vocabulary), path_or_buf=targetFile, sep='\t', header=None, index=False)
  
    
def frequency_bag_of_words(data,size):
    change = sparse.lil_matrix((data.shape[0], size))
    for index, row in data.iterrows():
        if type(row[0]) == type('str'):
            split = row[0].split(' ')
            for number in split:
                try:
                    change[index, int(number) - 1] += (1/len(split))
                except ValueError:
                    pass
    return change.tocsr() 

def binary_bag_of_words(data,size): 
    change = sparse.lil_matrix((data.shape[0], size))
    for index, row in data.iterrows():
        if type(row[0]) == type('str'):
            for number in row[0].split(' '):
                try:
                    change[index, int(number) - 1] = 1
                except ValueError:
                    pass
    return change.tocsr()


## Below code initalizes all the datasets used and creates vocabulary list and tranlsates text files for yelp reviews

In [7]:
yelp_train=pd.read_csv("yelp-train.txt", sep='\t', header=None)
yelp_test=pd.read_csv("yelp-test.txt", sep='\t', header=None)
yelp_valid=pd.read_csv("yelp-valid.txt", sep='\t', header=None)


yelp_vocab = vocabulary_list(yelp_train,10000)
yelp_vocab_file = makeFile("yelp-vocab.txt",yelp_vocab)

makeTranslation(yelp_train,"yelp-vocab.txt","yelp-train.txt",yelp_vocab)
makeTranslation(yelp_test,"yelp-vocab.txt","yelp-test.txt",yelp_vocab)
makeTranslation(yelp_valid,"yelp-vocab.txt","yelp-valid.txt",yelp_vocab)


print("done")

done


## Below code initalizes binary bag of words and frequency bag of words for yelp review text files

In [8]:
yelp_train=pd.read_csv("yelp-train.txt", sep='\t', header=None, encoding='ISO-8859-1', dtype=None)
yelp_test=pd.read_csv("yelp-test.txt", sep='\t', header=None, encoding='ISO-8859-1', dtype=None)
yelp_valid=pd.read_csv("yelp-valid.txt", sep='\t', header=None, encoding='ISO-8859-1', dtype=None)

binary_yelp_train=binary_bag_of_words(yelp_train,10000)
binary_yelp_test=binary_bag_of_words(yelp_test,10000)
binary_yelp_valid=binary_bag_of_words(yelp_valid,10000)

frequency_yelp_train=frequency_bag_of_words(yelp_train,10000)
frequency_yelp_test=frequency_bag_of_words(yelp_test,10000)
frequency_yelp_valid=frequency_bag_of_words(yelp_valid,10000)

print("done")

done


## A method, given a model and parameters, outputs the model with the best parammeter and it's F score for valid, it is trained on the training set

## Contains Model training for Naive Baiyes, Decision Trees, Linear Support Vector Machine, and other non specific classifications

In [9]:
# a method given a model and specified model used, runs the model with a list of paramters and outputs best f1 measure
def model_training(model, train, train_cols, valid, valid_cols, test, test_cols,
                   parameters=None, model_used=""):
    # when linear svm, naieve bayes, or decision tree isnt specified
    if model != None or parameters == None:
        model.fit(train, train_cols)
        prediction = pd.DataFrame(model.predict(train)).astype(int)
        print (model) 
        print("_________________________________________")     
        print("F1 Score(Train): ", f1_score(train_cols.astype(int), prediction, average='micro'))      
        prediction = pd.DataFrame(model.predict(valid)).astype(int)
        print("F1 Score(Valid): ", f1_score(valid_cols.astype(int), prediction, average='micro')) 
        prediction = pd.DataFrame(model.predict(test)).astype(int)
        print("F1 Score(Test): ", f1_score(test_cols.astype(int), prediction, average='micro'))
        print("\n")
        return model
    output_model = None
    output_f1 = -1
    
    # when the model is naieves bayes
    if model_used == "BernoulliNaiveBayes" or model_used == "MultinomialNaiveBayes":

        for alpha in parameters.get('alpha'):
            for fit_prior in parameters.get('fit_prior'):
                if model_used == "BernoulliNaiveBayes":
                    model = BernoulliNB(alpha=alpha, fit_prior=fit_prior)
                else:
                    model = MultinomialNB(alpha=alpha, fit_prior=fit_prior)

                model.fit(train, train_cols)
                temp = pd.DataFrame(model.predict(valid)).astype(int)
                new_f1 = f1_score(valid_cols.astype(int), temp, average='weighted')

                if output_f1 < new_f1:
                    output_f1 = new_f1
                    output_model = model
    
    #when the model is a decision tree
    if model_used == "DecisionTree":
        for criteria in parameters.get("criteria"):
            for splitter in parameters.get("splitter"):
                for class_weight in parameters.get("class_weight"):
                    for min_samples_split in parameters.get("min_samples_split"):
                        for min_samples_leaf in parameters.get("min_samples_leaf"):
                            for max_depth in parameters.get("max_depth"):
                                model = DecisionTreeClassifier(class_weight=class_weight, criterion=criteria,
                                                               splitter=splitter, min_samples_leaf=min_samples_leaf,
                                                               min_impurity_split=min_samples_split, max_depth=max_depth)
                                model.fit(train, train_cols)
                                temp = pd.DataFrame(model.predict(valid)).astype(int)
                                new_f1 = f1_score(valid_cols.astype(int), temp, average='weighted')

                                if output_f1 < new_f1:
                                    output_f1 = new_f1
                                    output_model = model

    #when the model is a linear support vector machine
    if model_used == "LinearSVC":
            for penalty in parameters.get("penalty"):
                for loss in parameters.get("loss"):
                    # Necessary because the combination of l1 loss and a penalty is not doable
                    if penalty == 'l1' and (loss == 'hinge' or loss == 'squared_hinge'):
                        continue
                    for tol in parameters.get("tol"):
                        for C in parameters.get("C"):
                            for max_iter in parameters.get("max_iter"):
                                for class_weight in parameters.get("class_weight"):
                                    model = LinearSVC(penalty=penalty, loss=loss, tol=tol, C=C, max_iter=max_iter,
                                                      class_weight=class_weight)
                                    model.fit(train, train_cols)
                                    temp = pd.DataFrame(model.predict(valid)).astype(int)
                                    new_f1 = f1_score(valid_cols.astype(int), temp, average='weighted')

                                    if output_f1 < new_f1:
                                        output_f1 = new_f1
                                        output_model = model
 
    print (output_model)
    print("_________________________________________")   
    prediction = pd.DataFrame(output_model.predict(valid)).astype(int)
    print("F1 Score(Valid): ", f1_score(valid_cols.astype(int), prediction, average='micro'))
    print("\n")
    return output_model



# Question 2: Model Training for  Yelp Review(Binary Bag of Words)

## As the assignment specified, many models with different paramters, i ran uniform random classifier, and majority classifier, naive bayes, decision tree, and finally linear support vector machine, we do this for yelp binary bag of words

In [10]:
#Uniform Random Classifier
yelp_uniform_random_classifier = DummyClassifier(strategy='uniform')
yelp_uniform_random_classifier = model_training(yelp_uniform_random_classifier, binary_yelp_train, yelp_train[1], binary_yelp_valid,
              yelp_valid[1], binary_yelp_test, yelp_test[1])


#Majority Class Classifier
yelp_majority_classifier = DummyClassifier(strategy='most_frequent')

yelp_majority_classifier = model_training(yelp_majority_classifier, binary_yelp_train, yelp_train[1], binary_yelp_valid,
              yelp_valid[1], binary_yelp_test, yelp_test[1])


#Naive Bayes with hyperparameters
naive_bayes_params = dict(alpha=np.array([100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]),
                          fit_prior=[True, False])

yelp_binary_NBC = model_training(None, binary_yelp_train, yelp_train.loc[:, [1]], binary_yelp_valid,
              yelp_valid.loc[:, [1]], binary_yelp_test, yelp_test.loc[:, [1]],
              parameters=naive_bayes_params, model_used="BernoulliNaiveBayes")



reduced_naive_bayes_params = dict(alpha=np.arange(0.002, 0.099, 0.001), fit_prior=[False])

yelp_binary_NBC = model_training(None, binary_yelp_train, yelp_train.loc[:, [1]], binary_yelp_valid,
              yelp_valid.loc[:, [1]], binary_yelp_test, yelp_test.loc[:, [1]],
              parameters=reduced_naive_bayes_params, model_used="BernoulliNaiveBayes")


#Decision Tree with hyperparameters
tree_params = dict(criteria=['gini', 'entropy'], splitter=['best', 'random'], max_depth=list(range(1, 10)),
                            min_samples_split= list(range(2, 10)), min_samples_leaf=list(range(1, 10)), class_weight=["balanced", None])

yelp_binary_DTC = model_training(None, binary_yelp_train, yelp_train.loc[:, [1]], binary_yelp_valid,
              yelp_valid.loc[:, [1]], binary_yelp_test, yelp_test.loc[:, [1]],
              parameters=tree_params, model_used="DecisionTree")


reduced_tree_params = dict(criteria=['entropy'], splitter=['best'], max_depth=list(range(1, 30)),
                            min_samples_split= list(range(2, 20)), min_samples_leaf=list(range(1, 20)), class_weight=[None])

yelp_binary_DTC = model_training(None, binary_yelp_train, yelp_train.loc[:, [1]], binary_yelp_valid,
              yelp_valid.loc[:, [1]], binary_yelp_test, yelp_test.loc[:, [1]], 
              parameters=reduced_tree_params, model_used="DecisionTree")



#Linear Support Vector Machine
linear_SVC_params = dict(penalty=['l1', 'l2'], loss=['hinge', 'squared_hinge'], class_weight=[None, 'balanced'],
                         tol=np.array([1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.0000001, 0.00000001]), max_iter =[1000],
                         C=([1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001,]))

yelp_binary_LSVC = model_training(None, binary_yelp_train, yelp_train.loc[:, [1]], binary_yelp_valid,
              yelp_valid.loc[:, [1]], binary_yelp_test, yelp_test.loc[:, [1]],
              parameters= linear_SVC_params, model_used="LinearSVC")



reduced_linear_SVC_params = dict(penalty=['l2'], loss=['squared_hinge'], class_weight=['balanced'],
                         tol=np.array([3, 2, 1]), max_iter=[1000, 10000], C=np.arange(0.0002, 0.0099, 0.0001))

yelp_binary_LSVC = model_training(None, binary_yelp_train, yelp_train.loc[:, [1]], binary_yelp_valid,
              yelp_valid.loc[:, [1]], binary_yelp_test, yelp_test.loc[:, [1]],
              parameters= reduced_linear_SVC_params, model_used="LinearSVC")

print("done")

DummyClassifier(constant=None, random_state=None, strategy='uniform')
_________________________________________
F1 Score(Train):  0.20614285714285713
F1 Score(Valid):  0.209
F1 Score(Test):  0.19850000000000004


DummyClassifier(constant=None, random_state=None, strategy='most_frequent')
_________________________________________
F1 Score(Train):  0.3525714285714286
F1 Score(Valid):  0.356
F1 Score(Test):  0.351


BernoulliNB(alpha=0.01, binarize=0.0, class_prior=None, fit_prior=False)
_________________________________________
F1 Score(Valid):  0.42999999999999994


BernoulliNB(alpha=0.01, binarize=0.0, class_prior=None, fit_prior=False)
_________________________________________
F1 Score(Valid):  0.42999999999999994


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=2,
            min_samples_leaf=6, min_samples_split=2,
            min_weight_fraction

# Question 3: Model Training for  Yelp Review(Frequency Bag of Words)

In [11]:

#Naive Bayes with hyperparameters
naive_bayes_params = dict(alpha=np.array([100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]),
                          fit_prior=[True, False])

yelp_freq_NBC = model_training(None, frequency_yelp_train, yelp_train.loc[:, [1]], frequency_yelp_valid,
              yelp_valid.loc[:, [1]], frequency_yelp_test, yelp_test.loc[:, [1]],
              parameters=naive_bayes_params, model_used="MultinomialNaiveBayes")



reduced_naive_bayes_params = dict(alpha=np.arange(0.002, 0.099, 0.001), fit_prior=[False])

yelp_freq_NBC = model_training(None, frequency_yelp_train, yelp_train.loc[:, [1]], frequency_yelp_valid,
              yelp_valid.loc[:, [1]], frequency_yelp_test, yelp_test.loc[:, [1]],
              parameters=reduced_naive_bayes_params, model_used="MultinomialNaiveBayes")


#Decision Tree with hyperparameters
tree_params = dict(criteria=['gini', 'entropy'], splitter=['best', 'random'], max_depth=list(range(1, 10)),
                            min_samples_split= list(range(2, 10)), min_samples_leaf=list(range(1, 10)), class_weight=["balanced", None])

yelp_freq_DTC = model_training(None, frequency_yelp_train, yelp_train.loc[:, [1]], frequency_yelp_valid,
              yelp_valid.loc[:, [1]], frequency_yelp_test, yelp_test.loc[:, [1]],
              parameters=tree_params, model_used="DecisionTree")


reduced_tree_params = dict(criteria=['entropy'], splitter=['best'], max_depth=list(range(1, 30)),
                            min_samples_split= list(range(2, 20)), min_samples_leaf=list(range(1, 20)), class_weight=[None])

yelp_freq_DTC = model_training(None, frequency_yelp_train, yelp_train.loc[:, [1]], frequency_yelp_valid,
              yelp_valid.loc[:, [1]], frequency_yelp_test, yelp_test.loc[:, [1]], 
              parameters=reduced_tree_params, model_used="DecisionTree")



#Linear Support Vector Machine
linear_SVC_params = dict(penalty=['l1', 'l2'], loss=['hinge', 'squared_hinge'], class_weight=[None, 'balanced'],
                         tol=np.array([1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.0000001, 0.00000001]), max_iter =[1000],
                         C=([1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001,]))

yelp_freq_LSVC = model_training(None, frequency_yelp_train, yelp_train.loc[:, [1]], frequency_yelp_valid,
              yelp_valid.loc[:, [1]], frequency_yelp_test, yelp_test.loc[:, [1]],
              parameters= linear_SVC_params, model_used="LinearSVC")



reduced_linear_SVC_params = dict(penalty=['l2'], loss=['squared_hinge'], class_weight=['balanced'],
                         tol=np.array([3, 2, 1]), max_iter=[1000, 10000], C=np.arange(0.0002, 0.0099, 0.0001))

yelp_freq_LSVC = model_training(None, frequency_yelp_train, yelp_train.loc[:, [1]], frequency_yelp_valid,
              yelp_valid.loc[:, [1]], frequency_yelp_test, yelp_test.loc[:, [1]],
              parameters= reduced_linear_SVC_params, model_used="LinearSVC")

print("done")


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=False)
_________________________________________
F1 Score(Valid):  0.508


MultinomialNB(alpha=0.01, class_prior=None, fit_prior=False)
_________________________________________
F1 Score(Valid):  0.508


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=9,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=2,
            min_samples_leaf=7, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
_________________________________________
F1 Score(Valid):  0.40599999999999997


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=19,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=2,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state

# Question 4: Model Training for  IMDB Review(Binary Bag of Words)

## Below code initalizes all the datasets used and creates vocabulary list and tranlsates text files for IMDB reviews

In [12]:
IMDB_train=pd.read_csv("IMDB-train.txt", sep='\t', header=None)
IMDB_test=pd.read_csv("IMDB-test.txt", sep='\t', header=None)
IMDB_valid=pd.read_csv("IMDB-valid.txt", sep='\t', header=None)


IMDB_vocab = vocabulary_list(IMDB_train,10000)
IMDB_vocab_file = makeFile("IMDB-vocab.txt",IMDB_vocab)

makeTranslation(IMDB_train,"IMDB-vocab.txt","IMDB-train.txt",IMDB_vocab)
makeTranslation(IMDB_test,"IMDB-vocab.txt","IMDB-test.txt",IMDB_vocab)
makeTranslation(IMDB_valid,"IMDB-vocab.txt","IMDB-valid.txt",IMDB_vocab)


print("done")

done


In [13]:
IMDB_train=pd.read_csv("IMDB-train.txt", sep='\t', header=None, encoding='ISO-8859-1', dtype=None)
IMDB_test=pd.read_csv("IMDB-test.txt", sep='\t', header=None, encoding='ISO-8859-1', dtype=None)
IMDB_valid=pd.read_csv("IMDB-valid.txt", sep='\t', header=None, encoding='ISO-8859-1', dtype=None)

binary_IMDB_train=binary_bag_of_words(IMDB_train,10000)
binary_IMDB_test=binary_bag_of_words(IMDB_test,10000)
binary_IMDB_valid=binary_bag_of_words(IMDB_valid,10000)


frequency_IMDB_train=frequency_bag_of_words(IMDB_train,10000)
frequency_IMDB_test=frequency_bag_of_words(IMDB_test,10000)
frequency_IMDB_valid=frequency_bag_of_words(IMDB_valid,10000)



print("done")

done


In [14]:
#Uniform Random Classifier
IMDB_uniform_random_classifier = DummyClassifier(strategy='uniform')
IMDB_uniform_random_classifier = model_training(IMDB_uniform_random_classifier, binary_IMDB_train, IMDB_train[1], binary_IMDB_valid,
              IMDB_valid[1], binary_IMDB_test, IMDB_test[1])


#Majority Class Classifier
IMDB_majority_classifier = DummyClassifier(strategy='most_frequent')

IMDB_majority_classifier = model_training(IMDB_majority_classifier, binary_IMDB_train, IMDB_train[1], binary_IMDB_valid,
              IMDB_valid[1], binary_IMDB_test, IMDB_test[1])


#Naive Bayes with hyperparameters
naive_bayes_params = dict(alpha=np.array([100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]),
                          fit_prior=[True, False])

IMDB_binary_NBC = model_training(None, binary_IMDB_train, IMDB_train.loc[:, [1]], binary_IMDB_valid,
              IMDB_valid.loc[:, [1]], binary_IMDB_test, IMDB_test.loc[:, [1]],
              parameters=naive_bayes_params, model_used="BernoulliNaiveBayes")



reduced_naive_bayes_params = dict(alpha=np.arange(0.002, 0.099, 0.001), fit_prior=[False])

IMDB_binary_NBC = model_training(None, binary_IMDB_train, IMDB_train.loc[:, [1]], binary_IMDB_valid,
              IMDB_valid.loc[:, [1]], binary_IMDB_test, IMDB_test.loc[:, [1]],
              parameters=reduced_naive_bayes_params, model_used="BernoulliNaiveBayes")


#Decision Tree with hyperparameters
tree_params = dict(criteria=['gini', 'entropy'], splitter=['best', 'random'], max_depth=list(range(1, 10)),
                            min_samples_split= list(range(2, 10)), min_samples_leaf=list(range(1, 10)), class_weight=["balanced", None])

IMDB_binary_DTC = model_training(None, binary_IMDB_train, IMDB_train.loc[:, [1]], binary_IMDB_valid,
              IMDB_valid.loc[:, [1]], binary_IMDB_test, IMDB_test.loc[:, [1]],
              parameters=tree_params, model_used="DecisionTree")


reduced_tree_params = dict(criteria=['entropy'], splitter=['best'], max_depth=list(range(1, 30)),
                            min_samples_split= list(range(2, 20)), min_samples_leaf=list(range(1, 20)), class_weight=[None])

IMDB_binary_DTC = model_training(None, binary_IMDB_train, IMDB_train.loc[:, [1]], binary_IMDB_valid,
              IMDB_valid.loc[:, [1]], binary_IMDB_test, IMDB_test.loc[:, [1]], 
              parameters=reduced_tree_params, model_used="DecisionTree")



#Linear Support Vector Machine
linear_SVC_params = dict(penalty=['l1', 'l2'], loss=['hinge', 'squared_hinge'], class_weight=[None, 'balanced'],
                         tol=np.array([1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.0000001, 0.00000001]), max_iter =[1000],
                         C=([1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.0000001, 0.00000001]))

IMDB_binary_LSVC = model_training(None, binary_IMDB_train, IMDB_train.loc[:, [1]], binary_IMDB_valid,
              IMDB_valid.loc[:, [1]], binary_IMDB_test, IMDB_test.loc[:, [1]],
              parameters= linear_SVC_params, model_used="LinearSVC")



reduced_linear_SVC_params = dict(penalty=['l2'], loss=['squared_hinge'], class_weight=['balanced'],
                         tol=np.array([3, 2, 1]), max_iter=[1000, 10000], C=np.arange(0.0002, 0.0099, 0.0001))

IMDB_binary_LSVC = model_training(None, binary_IMDB_train, IMDB_train.loc[:, [1]], binary_IMDB_valid,
              IMDB_valid.loc[:, [1]], binary_IMDB_test, IMDB_test.loc[:, [1]],
              parameters= reduced_linear_SVC_params, model_used="LinearSVC")

print("done")

DummyClassifier(constant=None, random_state=None, strategy='uniform')
_________________________________________
F1 Score(Train):  0.5044666666666666
F1 Score(Valid):  0.4939
F1 Score(Test):  0.503


DummyClassifier(constant=None, random_state=None, strategy='most_frequent')
_________________________________________
F1 Score(Train):  0.5
F1 Score(Valid):  0.5
F1 Score(Test):  0.5


BernoulliNB(alpha=0.01, binarize=0.0, class_prior=None, fit_prior=True)
_________________________________________
F1 Score(Valid):  0.8432


BernoulliNB(alpha=0.014, binarize=0.0, class_prior=None, fit_prior=False)
_________________________________________
F1 Score(Valid):  0.8434


DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=2,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           

# Question 5: Model Training for  IMDB Review(Frequency Bag of Words)

In [15]:

#Naive Bayes with hyperparameters
naive_bayes_params = dict(alpha=np.array([100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]),
                          fit_prior=[True, False])

IMDB_freq_NBC = model_training(None, frequency_IMDB_train, IMDB_train.loc[:, [1]], frequency_IMDB_valid,
              IMDB_valid.loc[:, [1]], frequency_IMDB_test, IMDB_test.loc[:, [1]],
              parameters=naive_bayes_params, model_used="MultinomialNaiveBayes")



reduced_naive_bayes_params = dict(alpha=np.arange(0.002, 0.099, 0.001), fit_prior=[False])


IMDB_freq_NBC = model_training(None, frequency_IMDB_train, IMDB_train.loc[:, [1]], frequency_IMDB_valid,
              IMDB_valid.loc[:, [1]], frequency_IMDB_test, IMDB_test.loc[:, [1]],
              parameters=reduced_naive_bayes_params, model_used="MultinomialNaiveBayes")




#Decision Tree with hyperparameters
tree_params = dict(criteria=['gini', 'entropy'], splitter=['best', 'random'], max_depth=list(range(1, 10)),
                            min_samples_split= list(range(2, 10)), min_samples_leaf=list(range(1, 10)), class_weight=["balanced", None])

IMDB_freq_DTC = model_training(None, frequency_IMDB_train, IMDB_train.loc[:, [1]], frequency_IMDB_valid,
              IMDB_valid.loc[:, [1]], frequency_IMDB_test, IMDB_test.loc[:, [1]],
              parameters=tree_params, model_used="DecisionTree")


reduced_tree_params = dict(criteria=['entropy'], splitter=['best'], max_depth=list(range(1, 30)),
                            min_samples_split= list(range(2, 20)), min_samples_leaf=list(range(1, 20)), class_weight=[None])

IMDB_freq_DTC = model_training(None, frequency_IMDB_train, IMDB_train.loc[:, [1]], frequency_IMDB_valid,
              IMDB_valid.loc[:, [1]], frequency_IMDB_test, IMDB_test.loc[:, [1]], 
              parameters=reduced_tree_params, model_used="DecisionTree")



#Linear Support Vector Machine
linear_SVC_params = dict(penalty=['l1', 'l2'], loss=['hinge', 'squared_hinge'], class_weight=[None, 'balanced'],
                         tol=np.array([1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.0000001, 0.00000001]), max_iter =[1000],
                         C=([1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.0000001, 0.00000001]))

IMDB_freq_LSVC = model_training(None, frequency_IMDB_train, IMDB_train.loc[:, [1]], frequency_IMDB_valid,
              IMDB_valid.loc[:, [1]], frequency_IMDB_test, IMDB_test.loc[:, [1]],
              parameters= linear_SVC_params, model_used="LinearSVC")



reduced_linear_SVC_params = dict(penalty=['l2'], loss=['squared_hinge'], class_weight=['balanced'],
                         tol=np.array([3, 2, 1]), max_iter=[1000, 10000], C=np.arange(0.0002, 0.0099, 0.0001))

IMDB_freq_LSVC = model_training(None, frequency_IMDB_train, IMDB_train.loc[:, [1]], frequency_IMDB_valid,
              IMDB_valid.loc[:, [1]], frequency_IMDB_test, IMDB_test.loc[:, [1]],
              parameters= reduced_linear_SVC_params, model_used="LinearSVC")
print("done")

MultinomialNB(alpha=0.0001, class_prior=None, fit_prior=True)
_________________________________________
F1 Score(Valid):  0.8495


MultinomialNB(alpha=0.002, class_prior=None, fit_prior=False)
_________________________________________
F1 Score(Valid):  0.8496


DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=2,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
_________________________________________
F1 Score(Valid):  0.5


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=2,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
    